# 0. Khai báo các thư viện

In [1]:
import pandas as pd
from scipy import stats
import math

# 1. Đọc dữ liệu từ file CSV

In [2]:
df = pd.read_csv("https://s3.ap-southeast-1.amazonaws.com/learn.codegym.vn/media/public/2022/01/08185827/house_price_Do%CC%82%CC%81ng-Da_Ha%CC%80-No%CC%A3%CC%82i_subdata.csv", encoding = "ISO-8859-1")
df.head()

,area,price,postDate,land_certificate,house_direction,balcony_direction,toilet,bedroom,floor,lat,long,property_type
0,50.0,14700.0,44197,So do,NaN,NaN,4.0,6.0,6.0,21.015698,105.824583,trong ngo
1,57.0,4200.0,44265,So do,Nam,Nam,3.0,8.0,4.0,21.025208,105.833111,trong ngo
2,62.0,11000.0,44268,So do,NaN,dong-Nam,6.0,5.0,6.0,21.025379,105.801011,trong ngo
3,26.0,2900.0,44292,So do,NaN,NaN,4.0,3.0,4.0,21.014325,105.834918,trong ngo
4,40.0,3950.0,44292,So do,NaN,NaN,NaN,NaN,5.0,21.009862,105.838083,trong ngo


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 671 entries, 0 to 670
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   area               665 non-null    float64
 1   price              617 non-null    float64
 2   postDate           671 non-null    int64  
 3   land_certificate   369 non-null    object 
 4   house_direction    42 non-null     object 
 5   balcony_direction  24 non-null     object 
 6   toilet             240 non-null    float64
 7   bedroom            320 non-null    float64
 8   floor              366 non-null    float64
 9   lat                671 non-null    float64
 10  long               671 non-null    float64
 11  property_type      671 non-null    object 
dtypes: float64(7), int64(1), object(4)
memory usage: 63.0+ KB


# 2. Giữa giá nhà và diện tích có tương quan với nhau

In [4]:
df1 = df.loc[:, ["price", "area"]]
df1.dropna(inplace = True)

Do giá nhà và diện tích là hai thuộc tính định lượng nên sử dụng kiểm định Pearson
- Giả thuyết không: Giá nhà và diện tích không có tương quan với nhau
- Giả thuyết đối: Giá nhà và diện tích có tương quan với nhau

In [5]:
stats.pearsonr(df1.price, df1.area)

(0.2285210190679815, 9.947613326077628e-09)

Do pvalue ~ 0 < 0.05, nhưng giá trị r = 0.22 nên có tương quan tuyến tính yếu

# 3. Giữa giá nhà và tọa độ địa lý (lat, long) có tương quan với nhau

In [6]:
df2 = df.loc[:, ["price", "long", "lat"]]
df2.dropna(inplace = True)

In [7]:
df2.long.describe()

count    617.000000
mean     105.825342
std        0.009132
min      105.795544
25%      105.822425
50%      105.828921
75%      105.829949
max      105.841407
Name: long, dtype: float64

In [8]:
df2.lat.describe()

count    617.000000
mean      21.015482
std        0.006604
min       20.998499
25%       21.009862
50%       21.017799
75%       21.018995
max       21.036845
Name: lat, dtype: float64

In [9]:
# Biến đổi toạ độ thành giá trị khoảng cách.
df2["distance"] = (df2.long ** 2 + df2.lat ** 2) ** 0.5

In [10]:
stats.pearsonr(df2.price, df2.distance)

(0.06514579858354355, 0.10595941259379761)

Do pvalue = 0.1 > 0.05 nên không có tương quan với nhau

# 4. Giữa thuộc tính land_certificate và property_type có tương quan với nhau

In [11]:
df3 = df.loc[:, ["land_certificate", "property_type"]]
df3.fillna("Unknown", inplace = True)

In [12]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 671 entries, 0 to 670
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   land_certificate  671 non-null    object
 1   property_type     671 non-null    object
dtypes: object(2)
memory usage: 10.6+ KB


Do land_certificate và property_type là hai thuộc tính định tính nên sử dụng kiểm định Chi bình phương để kiểm định.
- Giả thuyết không: Hai thuộc tính không có tương quan vơi nhau
- Giả thuyết đối: hai thuộc tính có tương quan với nhau

In [13]:
contigency = pd.crosstab(df3.land_certificate, df3.property_type)
contigency

property_type,mat pho,trong ngo
land_certificate,,
So do,114,255
Unknown,108,194


In [14]:
print(stats.chi2_contingency(contigency))

(1.564117781583915, 0.21106331497658828, 1, array([[122.08345753, 246.91654247],
       [ 99.91654247, 202.08345753]]))


Do pvalue = 0.2 > 0.05 nên không có tính tương quan giữa land_certificate và property_type

# 5. Hãy  mã hóa lại thuộc tính giá nhà theo đơn vị (nghìn đồng/m2) và sắp xếp giá nhà thành 4 mức tương ứng với các khoảng tứ phân vị. Tiến hành kiểm định tương quan giữa biến giá nhà này với các biến tọa độ vị trí (lat, long)

In [15]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 617 entries, 0 to 670
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   price     617 non-null    float64
 1   long      617 non-null    float64
 2   lat       617 non-null    float64
 3   distance  617 non-null    float64
dtypes: float64(4)
memory usage: 24.1 KB


In [16]:
# Tiến hành tính các khoảng tứ phân vị của price
q1, q2, q3 = df2.price.quantile(0.25), df2.price.quantile(0.5), df2.price.quantile(0.75)

In [17]:
# Biến đổi Price thành cat
def change_price_cat(series):
    if series < q1:
        return 1
    elif q1 <= series < q2:
        return 2
    elif q2 <= series < q3:
        return 3
    else:
        return 4

df2["price_cat"] = df2.price.apply(change_price_cat)

In [18]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 617 entries, 0 to 670
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   price      617 non-null    float64
 1   long       617 non-null    float64
 2   lat        617 non-null    float64
 3   distance   617 non-null    float64
 4   price_cat  617 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 28.9 KB


Do price_cat là thuộc tính thứ bậc và distance là thuộc tính liên tục nên sử dụng kiểm định spearman
- Giả thuyết không: price_cat và distance không tương quan với nhau
- Giả thuyết đối: price_cat và distance có liên quan với nhau

In [19]:
stats.spearmanr(df2.price_cat, df2.distance)

SpearmanrResult(correlation=-0.20389996449656927, pvalue=3.2513963641313107e-07)

Do pvalue ~ 0 < 0.05 nên hai thuộc tính có tương quan và tương quan tuyến tính yếu